### Task 0 Before your go

> 1. Rename Assignment-02-###.ipynb where ### is your student ID.
> 2. The deadline of Assignment-02 is 23:59pm, 04-21-2024
> 3. In this assignment, you will use word embeddings to explore our Wikipedia dataset.

### Task 1 Train word embeddings using SGNS 
> Use our enwiki-train.json as training data. You can use the [Gensim tool](https://radimrehurek.com/gensim/models/word2vec.html). But it is recommended to implement by yourself. You should explain how hyper-parameters such as dimensionality of embeddings, window size, the parameter of negative sampling strategy, and initial learning rate have been chosen.

In [38]:
# import some necessary libraries
from typing import List
from collections import defaultdict
from gensim.models import Word2Vec, Doc2Vec
from gensim import utils
import json
import random
import math
import numpy as np
import matplotlib.pyplot as plt
import heapq

In [21]:
# load the train and test data from the json file

# NOTE: The function is inherited from my solution of assignment 1
def load_json(file_path: str) -> list:
    """
    Fetch the data from `.json` file and concat them into a list.

    Input:
    - file_path: The relative file path of the `.json` file

    Returns:
    - join_data_list: A list containing the data, with the format of [{'title':<>, 'label':<>, 'text':<>}, {}, ...]
    """
    join_data_list = []
    with open(file_path, "r") as json_file:
        for line in json_file:
            line = line.strip()
            # guaranteen the line is not empty
            if line: 
                join_data_list.append(json.loads(line))
    return join_data_list

train_file_path, test_file_path = "enwiki-train.json", "enwiki-test.json"
train_data_list, test_data_list = map(load_json, [train_file_path, test_file_path])

class Corpus:
    def __iter__(self):
        for line in train_data_list:
            yield utils.simple_preprocess(line["text"])


In [22]:
class MyWord2Vec:
    def __init__(self, text: List[map], dimensionality: int=100, window_size: int=5, negative_samples: int=5, lr: float=0.001) -> None:
        """
        Args:
        - text: The training data
        - dimensionality: The dimension of the word embeddings
        - window_size: The size of the context window
        - negative_samples: The number of negative samples
        - lr: Learning rate of the algorithm
        """
        self.dim = dimensionality
        self.window = window_size
        self.neg = negative_samples
        self.lr = lr
        self.__vocab = set()
        self.__word_frq = defaultdict(int)
        self.__word2idx = {}
        self.__idx2word = {}
        self.__embedding = None
        self.__context_words = []
        self.__context_targets = []
        self.__build(text)
        

    def __preprocess(self, text: List[str]) -> None:
        """
        Calculate the vocabulary and the frequency of each word in the training data, while maintaining the (idx, word) map.
        """
        for sample in text:
            words = sample["text"].split()
            for word in words:
                self.__vocab.add(word)
                self.__word_frq[word] += 1
        for idx, word in enumerate(self.__vocab):
            self.__word2idx[word] = idx
            self.__idx2word[idx] = word

    def __generate_training_data(self, text: List[str]) -> None:
        """
        Generate training data from each window and save them in `self.__context_words` and `self.__context_targets`
        """
        for sample in text:
            words = sample["text"].split()
            for i, curr_word in enumerate(words):
                # the "window" around the current world
                for j in range(max(0, i - self.window), min(i + self.window + 1, len(words))):
                    if i != j:
                        self.__context_words.append(self.__word2idx[curr_word])
                        self.__context_targets.append(self.__word2idx[words[j]])

    def __initialize_embedding(self) -> None:
        """
        Initialize the embedding matrix with random values
        """
        self.__embedding = np.random.uniform(-0.5 / self.dim, 0.5 / self.dim, size=(len(self.__vocab), self.dim))
    
    def __build(self, text: List[map]) -> None:
        """
        Compute and store the relevant information of the training data in the class
        """
        self.__preprocess(text)
        self.__generate_training_data(text)
        self.__initialize_embedding()

    def train(self, epochs: int=5) -> None: 
        for epoch in range(epochs):
            # learning rate decay
            learning_rate = self.lr * (1 - epoch / epochs)

            print("Training Epoch: %d" % (epoch + 1))

            for context_word, target_word in zip(self.__context_words, self.__context_targets):
                context_vector = self.__embedding[context_word]
                target_vector = self.__embedding[target_word]

                # positive sample update
                score = np.dot(target_vector, context_vector)
                exp_score = math.exp(score)
                grad_context = (exp_score / (1 + exp_score) - 1) * target_vector
                grad_target = (exp_score / (1 + exp_score) - 1) * context_vector
                self.__embedding[context_word] -= learning_rate * grad_context
                self.__embedding[target_word] -= learning_rate * grad_target

                # negative sample update
                for _ in range(self.neg):
                    negative_word = random.randint(0, len(self.__vocab) - 1)
                    if negative_word != target_word:
                        negative_vector = self.__embedding[negative_word]
                        score = np.dot(negative_vector, context_vector)
                        exp_score = math.exp(score)
                        grad_context = exp_score / (1 + exp_score) * negative_vector
                        grad_target = exp_score / (1 + exp_score) * context_vector
                        self.__embedding[context_word] -= learning_rate * grad_context
                        self.__embedding[target_word] -= learning_rate * grad_target


In [23]:
sentence = Corpus()
model = Word2Vec(sentences=sentence, vector_size=100, alpha=0.025, window=5, 
                               min_count=5, sample=0.001, seed=1, workers=3, min_alpha=0.0001, 
                               sg=1, negative=5, ns_exponent=0.75, epochs=5, sorted_vocab=1)

### Task 2 Find similar/dissimilar word pairs

> Randomly generate 100, 1000, and 10000-word pairs from the vocabularies. For each set, print 5 closest word pairs and 5 furthest word pairs (you can use cosine-similarity to measure two words). Explain your results.

In [78]:
def generate_random_paris(samples: int):
    """
    Generate random indices without replacement, then pairs the indices to get word pairs
    """
    indices = random.sample(range(len(model.wv)), 2 * samples)
    indices1, indices2 = indices[:samples], indices[samples:]
    return [model.wv.index_to_key[i] for i in indices1], [model.wv.index_to_key[i] for i in indices2]

def find_closest_furthest(num: int=5, words1: List[str]=None, words2: List[str]=None) -> None:
    """
    Find the cloest/furthest word pairs using `model.wv.similarity`.

    Here a heap queue is used to reduce time complexity to $O(n\log k)$, where k denotes the `num`
    """
    heap = []
    for i in range(len(words1)):
        # compute the similarity and push it into the heap
        heapq.heappush(heap, (model.wv.similarity(words1[i], words2[i]), words1[i], words2[i]))
    return heapq.nlargest(num, heap), heapq.nsmallest(num, heap)[::-1]

def print_word_pairs(results: List[tuple], flag: str) -> None:
    """
    Print the result in formatted string
    """
    print("The 5 {:>8} word pairs:".format(flag))
    for result in results:
       print("Word pairs: ({:>15}, {:>15}) --> Similarity: {:>8.6f}".format(result[1], result[2], result[0]))


random.seed(408)
pairs = [100, 1000, 10000]
for pair in pairs:
    print("For {:>5} random pairs from the vocabularies:".format(pair))
    cloest, furthest = find_closest_furthest(5, *generate_random_paris(pair))
    print_word_pairs(cloest, "closest")
    print_word_pairs(furthest, "furthest")
    print("-" * 70)

For   100 random pairs from the vocabularies:
The 5  closest word pairs:
Word pairs: (      herodotus,       sorcerers) --> Similarity: 0.888745
Word pairs: (           sown,     prohibitive) --> Similarity: 0.849635
Word pairs: (         shaded,           booby) --> Similarity: 0.839130
Word pairs: (        deflect,      comforting) --> Similarity: 0.830611
Word pairs: (        bruxing,        plumbers) --> Similarity: 0.827411
The 5 furthest word pairs:
Word pairs: (     violations,         yevgeny) --> Similarity: 0.248302
Word pairs: (     preferring,  neolissochilus) --> Similarity: 0.226008
Word pairs: (       superman,         outputs) --> Similarity: 0.219238
Word pairs: (        trapped,          arabia) --> Similarity: 0.195078
Word pairs: (     separately,       communism) --> Similarity: 0.124271
----------------------------------------------------------------------
For  1000 random pairs from the vocabularies:
The 5  closest word pairs:
Word pairs: (        itching,       

### Task 3 Present a document as an embedding

> For each document, you have several choices to generate document embedding: 1. Use the average of embeddings of all words in each document; 2. Use the first paragraph’s words and take an average on these embeddings; 3. Use the doc2vec algorithm to present each document. Do the above for both training and testing dataset

In [25]:
####################################################################################
###        1. Use the average of embeddings of all words in each document        ###
####################################################################################
doc_embeddings_1 = {}
for line in train_data_list:
    doc_title = line["title"]
    doc_embedding = sum(model.wv.n_similarity)



In [ ]:
####################################################################################
###  2. Use the first paragraph’s words and take an average on these embeddings  ###
####################################################################################

In [ ]:
####################################################################################
###             3. Use the doc2vec algorithm to present each document            ###
####################################################################################
model = Doc2Vec()

### Task 4 Build classifier to test docs
> Build softmax regression model to classifier testing documents based on these training doc embeddings. Does it getting better than Naive Bayes'? (You have 3 models.)

In [26]:
# Your code




### Task 5 Use t-SNE to project doc vectors

> Use t-SNE to project training document embeddings into 2d and plot them out for each of the above choices. Each point should have a specific color (represent a particular cluster). You may need to try different parameters of t-SNE. One can find more details about t-SNE in this [excellent article](https://distill.pub/2016/misread-tsne/).

In [27]:
# Your code


